In [1]:
import os
import torch
import numpy as np
import pandas as pd
import utils
import nids_models
import pickle
from nids_models import DNN
from utils import CustomDataset
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import joblib
import importlib
importlib.reload(utils)
importlib.reload(nids_models)

# Fix the kernel dead when load/save big file
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# Define the training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = train_loss / len(train_loader.dataset)
    accuracy = 100.0 * correct / total

    return train_loss, accuracy

device(type='cuda')

In [4]:
# Define the evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    test_loss = test_loss / len(test_loader.dataset)
    accuracy = 100.0 * correct / total

    return test_loss, accuracy, predictions, true_labels

In [5]:
# Set random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [6]:
# small_df = pd.read_csv(filename, nrows=100)

<class 'utils.CustomDataset'>


In [8]:
# Load the custom dataset
dataset = CustomDataset('datasets/nids/combined_data.csv')

129238

In [8]:
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True, pin_memory=True)

In [ ]:
# Split the dataset into train and test sets
train_size = 0.8
train_dataset, test_dataset = train_test_split(dataset, train_size=train_size, random_state=random_seed)

# Define the DataLoader for train and test sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
# Define model parameters
input_size = 1525  # Number of features in the dataset
hidden_size = 256
# num_classes = 4
num_classes = 2

In [11]:
save_dir

'./output_nids/dnn_fedprox/nids'

In [12]:
# Create the DNN model
model = DNN(input_size, hidden_size, num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10


# Move the model to the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    # Train the model
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    
    # Evaluate the model
    test_loss, test_accuracy, predictions, true_labels = evaluate(model, test_loader, criterion, device)
    
    # Print the training and evaluation metrics
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    
    # Generate classification report
    report = classification_report(true_labels, predictions)
    print('Classification Report:')
    print(report)

### Evaluate the global model all attacks type among clients

In [21]:
# Define the evaluation function
criterion = nn.CrossEntropyLoss()

def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    test_loss = test_loss / len(test_loader.dataset)
    accuracy = 100.0 * correct / total

    return test_loss, accuracy, predictions, true_labels

In [22]:
silo0_sub = CustomDataset('client_data/nids/0/0_test_new.csv')
silo1_sub = CustomDataset('client_data/nids/1/1_test_new.csv')
silo2_sub = CustomDataset('client_data/nids/2/2_test_new.csv')
silo3_sub = CustomDataset('client_data/nids/3/3_test_new.csv')

In [23]:
len(silo0_sub)

35077

In [24]:
test_silo0_sub = DataLoader(silo0_sub, batch_size=batch_size, shuffle=True)
test_silo1_sub = DataLoader(silo1_sub, batch_size=batch_size, shuffle=True)
test_silo2_sub = DataLoader(silo2_sub, batch_size=batch_size, shuffle=True)
test_silo3_sub = DataLoader(silo3_sub, batch_size=batch_size, shuffle=True)

In [25]:
len(test_silo0_sub)

1097

In [26]:
# Evaluate the model
test_loss, test_accuracy, predictions, true_labels = evaluate(glob_model, test_silo0_sub, criterion, device)

# Print the training and evaluation metrics
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Generate classification report
report = classification_report(true_labels, predictions)
print('Classification Report:')
print(report)

Test Loss: 0.0474, Test Accuracy: 98.26%
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     13566
           1       0.99      0.98      0.99     21511

    accuracy                           0.98     35077
   macro avg       0.98      0.98      0.98     35077
weighted avg       0.98      0.98      0.98     35077



In [27]:
# Evaluate the model
test_loss, test_accuracy, predictions, true_labels = evaluate(glob_model, test_silo1_sub, criterion, device)

# Print the training and evaluation metrics
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Generate classification report
report = classification_report(true_labels, predictions)
print('Classification Report:')
print(report)

Test Loss: 0.0264, Test Accuracy: 99.33%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     13464
           1       0.99      1.00      0.99     18283

    accuracy                           0.99     31747
   macro avg       0.99      0.99      0.99     31747
weighted avg       0.99      0.99      0.99     31747



In [28]:
# Evaluate the model
test_loss, test_accuracy, predictions, true_labels = evaluate(glob_model, test_silo2_sub, criterion, device)

# Print the training and evaluation metrics
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Generate classification report
report = classification_report(true_labels, predictions)
print('Classification Report:')
print(report)

Test Loss: 0.1962, Test Accuracy: 94.36%
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     13687
           1       0.99      0.90      0.94     13650

    accuracy                           0.94     27337
   macro avg       0.95      0.94      0.94     27337
weighted avg       0.95      0.94      0.94     27337



In [29]:
# Evaluate the model
test_loss, test_accuracy, predictions, true_labels = evaluate(glob_model, test_silo3_sub, criterion, device)

# Print the training and evaluation metrics
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Generate classification report
report = classification_report(true_labels, predictions)
print('Classification Report:')
print(report)

Test Loss: 0.2022, Test Accuracy: 94.22%
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     13591
           1       0.99      0.90      0.94     13746

    accuracy                           0.94     27337
   macro avg       0.95      0.94      0.94     27337
weighted avg       0.95      0.94      0.94     27337

